In [1]:
import numpy as np

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient()
db = client.cmv
posts_collection = db.posts
tl_comments_collection = db.tl_comments
deltad_replies_collection = db.deltad_replies

In [7]:
deltad_post_gen = posts_collection.find( {'tl_comment_delta_parents': {"$exists": True}})
undeltad_post_gen = posts_collection.find( {'tl_comment_delta_parents': {"$exists": False}})

In [8]:
deltad_post_list = list(deltad_post_gen)
undeltad_post_list = list(undeltad_post_gen)

In [9]:
print(len(deltad_post_list))
print(len(undeltad_post_list))

1146
5370


In [ ]:
deltad_tl_comment_gen =
undeltad_tl_comment_gen = 

In [10]:
#train test split
test_split_d = int(0.2*len(deltad_post_list))
test_split_u = int(0.2*len(undeltad_post_list))

np.random.shuffle(deltad_post_list)
np.random.shuffle(undeltad_post_list)

test_posts = deltad_post_list[0:test_split_d]
test_posts.extend(undeltad_post_list[0:test_split_u])

train_posts= deltad_post_list[test_split_d::]
train_posts.extend(undeltad_post_list[test_split_u::])

In [11]:
type(test_split_d-1)

int

In [13]:
test_post_labels = [1]*(test_split_d)
test_post_labels.extend(([0]*(test_split_u)))
test_post_labels = np.array(test_post_labels)

In [14]:
num_train_d = len(deltad_post_list) - test_split_d
num_train_u = len(undeltad_post_list) - test_split_u

train_post_labels = [1]*num_train_d
train_post_labels.extend([0]*(num_train_u))

train_post_labels = np.array(train_post_labels)

In [15]:
train_post_labels.shape

(5213,)

In [17]:
len(train_posts)

5213

In [18]:
len(test_posts)

1303

In [19]:
train_post_texts = [doc['post_text'] for doc in train_posts]
test_post_texts = [doc['post_text'] for doc in test_posts]

In [20]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import string

In [21]:
alist = ['a', 'b']
blist = ['c', 'd']
alist.extend(blist)
print(alist)

['a', 'b', 'c', 'd']


In [22]:
def clean_text(posts, tokenizer, stemmer):
    stemmer_inst = stemmer()
    tokenizer_inst = tokenizer()
    cleaned_posts = []
    for post in posts:
        #cleaned_words = []
        
        #strip punctuation and digits from whole post
        to_replace = [punc for punc in string.punctuation+string.digits]
        replacement_table = str.maketrans({key: None for key in to_replace})
        stripped_post = post.translate(replacement_table)
        
        #lower case post
        lowered_post = stripped_post.lower()
        cleaned_posts.append(lowered_post)
        #tokenized_post = tokenizer_inst.tokenize(stripped_post) 
        #for word in tokenized_post:
            #low_word = stemmer_inst.stem(word.lower())
            #cleaned_words.append(low_word)
            #remove stopwords?
            #if low_word not in stopwords:
            #    cleaned_words.append(low_word)
        # cleaned_posts.append(' '.join(cleaned_words))
    return cleaned_posts

In [23]:
stemmer = PorterStemmer
tokenizer = WhitespaceTokenizer

In [24]:
cleaned_train_posts = clean_text(train_post_texts, tokenizer, stemmer)
cleaned_test_posts = clean_text(test_post_texts, tokenizer, stemmer)

In [47]:
my_vectorizer = CountVectorizer(max_df=0.9, min_df=1,
                                max_features=1000,
                                stop_words='english')

In [48]:
vectorized_train_posts = my_vectorizer.fit_transform(cleaned_train_posts)
vectorized_test_posts = my_vectorizer.fit_transform(cleaned_test_posts)

## Unsupervised Topic Extraction

In [27]:
from sklearn.decomposition import TruncatedSVD

In [92]:
lsa_model = TruncatedSVD(n_components=150, random_state=40)

In [93]:
lsa_model.fit(vectorized_train_posts)

TruncatedSVD(algorithm='randomized', n_components=150, n_iter=5,
       random_state=40, tol=0.0)

In [94]:
reduced_train_posts = lsa_model.transform(vectorized_train_posts)
reduced_test_posts = lsa_model.transform(vectorized_test_posts)

In [95]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
display_topics(lsa_model,my_vectorizer.get_feature_names(),20) # We have to look at the topics before hand and then add the labels afterwards


Topic  0
parents, level, thoughts, jobs, drugs, immigration, love, wasnt, took, wear, benefits, parts, killing, thought, voter, responses, running, got, racist, changing

Topic  1
type, vs, level, color, rule, immigration, wont, drugs, history, took, original, jobs, end, thoughts, start, car, wear, english, changing, positions

Topic  2
type, vs, parents, color, rule, start, end, english, original, history, car, wins, wall, wanted, great, positions, standing, care, planet, regardless

Topic  3
wont, mean, wins, gay, seen, type, blame, won, color, main, sense, parents, mainly, push, situations, figure, rule, vs, english, votes

Topic  4
gun, great, benefits, responses, parts, seen, let, love, votes, mind, argue, value, guns, situations, standing, start, later, nation, responsibility, idea

Topic  5
thoughts, parts, seen, responses, yes, gun, push, type, sense, consequences, vs, votes, trump, benefits, guns, months, driving, voter, color, critical

Topic  6
thoughts, parts, yes, respons

## Supervised Classification

In [96]:
from sklearn.metrics import confusion_matrix, roc_auc_score

In [97]:
from sklearn.svm import SVC

In [98]:
svc_model = SVC(class_weight='balanced', random_state=42)
svc_model.fit(reduced_train_posts, train_post_labels)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [99]:
predicted_test_labels = svc_model.predict(reduced_test_posts)
conf_mat = confusion_matrix(test_post_labels, predicted_test_labels)
print(conf_mat)

[[885 189]
 [199  30]]


In [100]:
print('mean accuracy is {}'.format(svc_model.score(reduced_test_posts, test_post_labels)))

mean accuracy is 0.7022256331542595


In [76]:
#from sklearn.cluster import DBSCAN

In [77]:
#dbscan_model = DBSCAN(eps=0.5, min_samples=5, metric=’euclidean’, metric_params=None, algorithm=’auto’, leaf_size=30, p=None, n_jobs=1)

In [33]:
#from sklearn.linear_model import LogisticRegression

In [34]:
#logreg = LogisticRegression(random_state=42, class_weight='balanced')
#logreg.fit(reduced_train_posts, train_post_labels)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [35]:
#predicted_test_labels = logreg.predict(reduced_test_posts)

In [38]:
#print('mean accuracy is {}'.format(logreg.score(reduced_test_posts, test_post_labels)))
#conf_mat = confusion_matrix(test_post_labels, predicted_test_labels)
#print(conf_mat)

mean accuracy is 0.6362240982348427
[[787 287]
 [187  42]]
